# Introduction

In this notebook I will be introducing a problem and then use the foursquare API to help answer the problem. I will find the best coffee shop in each neighborhood in New York. So that I always know where to go to get a good cup of coffee in New York.

I will use the Foursquare API to find the ratings for each coffee shop then map the best one in each neighborhood using the data from newyork_data.json and the folium library. 

# Code 

In [11]:
import pandas as pd #import all the packages
import geocoder
import folium
import numpy as np
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import random
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

In [2]:
#!conda update -n base -c defaults conda

In [3]:
with open('nyu.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
#create map of New York using latitude and longitude values
latitude = 40.71455
longitude = -74.00712
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [32]:
#define url variables
Client_id = 'VC3SXCW1GYOWFLCIM2TJ4P1E4KDXJW10HUVJPXNXYGVLY0FR'
Client_secret = 'XTDP4E1LYVDRSFJ3AIB4T3B0ZKJ0YWNIHDIA1RHZYBWV2SQV'
VERSION = '20180604'
LIMIT = 300
latitude = 40.71455
longitude = -74.00712
search_query = 'Coffee'
radius = 5000



In [33]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(Client_id, Client_secret, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VC3SXCW1GYOWFLCIM2TJ4P1E4KDXJW10HUVJPXNXYGVLY0FR&client_secret=XTDP4E1LYVDRSFJ3AIB4T3B0ZKJ0YWNIHDIA1RHZYBWV2SQV&ll=40.71455,-74.00712&v=20180604&query=Coffee&radius=5000&limit=300'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc3117802a172002816c216'},
 'response': {'venues': [{'id': '49c79540f964a520af571fe3',
    'name': 'Blue Spoon Coffee Co.',
    'location': {'address': '76 Chambers St',
     'crossStreet': 'at Broadway',
     'lat': 40.714427584609766,
     'lng': -74.00685853301651,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.714427584609766,
       'lng': -74.00685853301651}],
     'distance': 25,
     'postalCode': '10007',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['76 Chambers St (at Broadway)',
      'New York, NY 10007',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1573065210',
 

In [35]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,49c79540f964a520af571fe3,76 Chambers St,...,"[76 Chambers St (at Broadway), New York, NY 10...","[{'label': 'display', 'lat': 40.71442758460976...",40.714428,-74.006859,NaN,10007,NY,Blue Spoon Coffee Co.,v-1573065210,NaN
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,583af456ce593d41644bd7a6,120 Lafayette St Frnt C,...,"[120 Lafayette St Frnt C, New York, NY 10013, ...","[{'label': 'display', 'lat': 40.71863256946489...",40.718633,-74.000367,NaN,10013,NY,Kaigo Coffee Room,v-1573065210,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,5cf965947564f7002c79aa21,1 Liberty Plz,...,"[1 Liberty Plz (Broadway), New York, NY 10006,...","[{'label': 'display', 'lat': 40.709554, 'lng':...",40.709554,-74.010576,NaN,10006,NY,For Five Coffee Roasters,v-1573065210,NaN
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,55f05a02498ea2939736f07a,65 Nassau St,...,"[65 Nassau St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70947645413928...",40.709476,-74.008546,NaN,10038,NY,Proof Coffee Roaster,v-1573065210,139571223
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",1043479,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/gregorys-coffee-...,False,53051e8b498e331dc90d6eb4,42 Broadway,...,"[42 Broadway (btwn Exchange Pl & Morris St), N...","[{'label': 'display', 'lat': 40.70611, 'lng': ...",40.706110,-74.012850,Financial District,10004,NY,Gregorys Coffee,v-1573065210,440465559


In [36]:
# keeping only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Blue Spoon Coffee Co.,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",76 Chambers St,US,New York,United States,at Broadway,25,"[76 Chambers St (at Broadway), New York, NY 10...","[{'label': 'display', 'lat': 40.71442758460976...",40.714428,-74.006859,NaN,10007,NY,49c79540f964a520af571fe3
1,Kaigo Coffee Room,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",120 Lafayette St Frnt C,US,New York,United States,NaN,728,"[120 Lafayette St Frnt C, New York, NY 10013, ...","[{'label': 'display', 'lat': 40.71863256946489...",40.718633,-74.000367,NaN,10013,NY,583af456ce593d41644bd7a6
2,For Five Coffee Roasters,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",1 Liberty Plz,US,New York,United States,Broadway,627,"[1 Liberty Plz (Broadway), New York, NY 10006,...","[{'label': 'display', 'lat': 40.709554, 'lng':...",40.709554,-74.010576,NaN,10006,NY,5cf965947564f7002c79aa21
3,Proof Coffee Roaster,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",65 Nassau St,US,New York,United States,NaN,577,"[65 Nassau St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70947645413928...",40.709476,-74.008546,NaN,10038,NY,55f05a02498ea2939736f07a
4,Gregorys Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",42 Broadway,US,New York,United States,btwn Exchange Pl & Morris St,1056,"[42 Broadway (btwn Exchange Pl & Morris St), N...","[{'label': 'display', 'lat': 40.70611, 'lng': ...",40.706110,-74.012850,Financial District,10004,NY,53051e8b498e331dc90d6eb4


In [37]:
#mapping the venues
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) 
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.Marker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map

# Conclusion

After mapping coffee shops in New York it is safe to say there are a lot of coffee shops in the area